In [1]:
from pydantic import SecretStr
from pydantic_settings import BaseSettings, SettingsConfigDict
import os
from langchain_core.output_parsers import JsonOutputParser
from langchain.prompts import PromptTemplate
from langchain_core.pydantic_v1 import BaseModel, Field
from langchain_openai import ChatOpenAI
from typing import List


class Settings(BaseSettings):
    model_config = SettingsConfigDict(env_file='input',
                                      env_file_encoding='utf-8')
    openai_api_key: SecretStr

config = Settings()
os.environ["OPENAI_API_KEY"] = config.openai_api_key.get_secret_value()

## JsonOutputParser

In [2]:
class About(BaseModel):
    name: str = Field(description="Write the author's full name.")
    dob: str = Field(description="Write down the author's date of birth.")
    education: str = Field(description="Describes the author's educational experience / background.")
    work: List[str] = Field(description="Describes the author's work experience / background.")


parser = JsonOutputParser(pydantic_object=About)
prompt = PromptTemplate(
    input_variables=["query"],
    template="Answer the user query.\n"
             "{format_instructions}\n"
             "{content_}.\n"
             "{query}\n",
    partial_variables={"format_instructions": parser.get_format_instructions(),
                       "content_": " "}, ) # доп. контент, если нужно

model = ChatOpenAI(temperature=0)
chain = prompt | model | parser


# Текст с лишней информацией
query = """
Меня зовут Иван Иванович Пупкин.
Птицы в небе парят, парят и поют,
Их голоса звучат над зеленым лугом.
Природа величественна и нежна,
В ней сокрыты тайны и красоты много.
Дата рождения - 10.10.2000.
В лесу тишина, покой и свежесть,
Где ветер шепчет свои нежные слова.
Зеленые деревья, ветви, листья,
Встречают утро своими объятьями.
Я имею высшее экономическое образование.
Мой опыт работы: менеджер фирмы Альфа в 2020 году,
Ручей мелодично поет песню свою,
Соприкасаясь с камнями и землей.
Цветы яркие расцветают в полях,
Радуя глаза своей красотой своей.
менеджер фирмы Бетта в 2021 году,
менеджер фирмы Тетта в 2023 году."""


# Из всего текста выбираем только то, что нужно из класса About
chain.invoke({"query": query})

{'name': 'Иван Иванович Пупкин',
 'dob': '10.10.2000',
 'education': 'Высшее экономическое образование',
 'work': ['Менеджер фирмы Альфа в 2020 году',
  'Менеджер фирмы Бетта в 2021 году',
  'Менеджер фирмы Тетта в 2023 году']}

 ---------------------------------------------------------------------------
## Парсер с использованием индексной базы данных (для больших объемов текста)
## Langchain output_parsers + llama_index

In [10]:
from llama_index.core import VectorStoreIndex, SimpleDirectoryReader
from llama_index.core.output_parsers import LangchainOutputParser
from llama_index.llms.openai import OpenAI
from langchain.output_parsers import StructuredOutputParser, ResponseSchema
import requests


def download_from_url(url, save_path):
    with open(save_path, 'wb') as f:
        f.write(requests.get(url).content)

url = 'https://raw.githubusercontent.com/run-llama/llama_index/main/docs/docs/examples/data/paul_graham/paul_graham_essay.txt'
download_from_url(url, 'data/paul_graham_essay.txt')

In [4]:
documents = SimpleDirectoryReader("data/").load_data()
index_db = VectorStoreIndex.from_documents(documents, chunk_size=1024) # индексная база
len(index_db.docstore.docs) # к-во чанков

22

In [5]:
schemas = [
    ResponseSchema(
        name="Education",
        description="Describes the author's educational experience / background."),
    ResponseSchema(
        name="Work",
        description="Describes the author's work experience / background."),
]

output_parser = StructuredOutputParser.from_response_schemas(schemas)
parser = LangchainOutputParser(output_parser)

In [6]:
from llama_index.core.prompts.default_prompts import (DEFAULT_TEXT_QA_PROMPT_TMPL)
print(parser.format(DEFAULT_TEXT_QA_PROMPT_TMPL))

Context information is below.
---------------------
{context_str}
---------------------
Given the context information and not prior knowledge, answer the query.
Query: {query_str}
Answer: 

The output should be a markdown code snippet formatted in the following schema, including the leading and trailing "```json" and "```":

```json
{{
	"Education": string  // Describes the author's educational experience / background.
	"Work": string  // Describes the author's work experience / background.
}}
```


In [7]:
llm = OpenAI(output_parser=parser)
query_engine = index_db.as_query_engine(llm=llm)

question = "What are a things the author did growing up?"
response = query_engine.query(question)
print('\n', response)


 {'Education': 'Studied philosophy in college before switching to AI', 'Work': 'Worked on writing short stories and programming, including writing simple games, a rocket prediction program, and a word processor'}


 -----------------------------------------------------------------------------
## Еще один вариант с индексной базой
## Langchain output_parsers + llama_index

In [11]:
import re

def download_from_url(url, save_path):
    match_ = re.search('/document/d/([a-zA-Z0-9-_]+)', url)
    if match_ is None:
        raise ValueError('Invalid Google Docs URL')
    doc_id = match_.group(1)
    response = requests.get(f'https://docs.google.com/document/d/{doc_id}/export?format=txt')
    response.raise_for_status()
    with open(save_path, 'w', encoding='utf-8') as f:
        f.write(response.text)

url = 'https://docs.google.com/document/d/1PGw2Y4PSJ2S7vfPlge9j6G_x0zX6jX4UEIitfwlP-c0/edit?usp=sharing'
download_from_url(url, 'data2/rules.txt')

In [13]:
documents = SimpleDirectoryReader('data2/').load_data()
index_db = VectorStoreIndex.from_documents(documents, chunk_size=512) # индексная база
len(index_db.docstore.docs) # к-во чанков

82

In [14]:
schemas = [
    ResponseSchema(name="Q1",
        description="Опиши Независимость Юрисдикционных органов"),
    ResponseSchema(name="Q2",
        description="Опиши Право на участие в заседании Юрисдикционного органа"),
    ResponseSchema(name="Q3",
        description="Опиши Общие начала применения спортивных санкций"),
]
output_parser = StructuredOutputParser.from_response_schemas(schemas)
parser = LangchainOutputParser(output_parser)

llm = OpenAI(output_parser=parser)
query_engine = index_db.as_query_engine(llm=llm)

question = "Опиши указанные понятия"
response = query_engine.query(question)
print('\n', response)


 {'Q1': 'Независимость Юрисдикционных органов означает, что они могут принимать решения без внешнего влияния и вмешательства, обеспечивая объективность и справедливость в своей деятельности.', 'Q2': 'Право на участие в заседании Юрисдикционного органа предоставляет заинтересованным лицам возможность присутствовать на заседаниях, высказывать свою позицию, представлять свидетельские показания или доказательства в свою защиту.', 'Q3': 'Общие начала применения спортивных санкций включают в себя принципы справедливости, пропорциональности, недискриминации и соблюдения процедурных правил при наложении наказаний за нарушения.'}
